# BERT Practical

Practical work on BERT for the course Natural Language Processing in M2 MoSIG

> Author: Archit YADAV

The final findings and experimentation details can be found in the last markdown section of this notebook titled "Report".

Table of Contents

1. IMPORT MODULES
2. DATA
3. TOKENISATION
4. MODEL
5. TRAINING
6. PREDICTIONS
7. IMPROVE THE MODEL

  REPORT

## 1.IMPORT MODULES

In [ ]:
!pip install transformers
!pip install torch

In [2]:
import torch
import transformers

# Managing arrays
import numpy as np

In [3]:
# load the TensorBoard notebook extension
# %load_ext tensorboard

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

GPU is available.


## 2.DATA

### 2.1 Downloading of Data

In [4]:
from sklearn.datasets import fetch_20newsgroups

categories = [
 'comp.windows.x',
 'sci.med',
 'soc.religion.christian',
 'talk.politics.guns',
]

# Download data if not already present in data_home
trainset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')
testset = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')

# Define input data and labels for training and testing
x_train = trainset.data
y_train = trainset.target
x_test = testset.data
y_test = testset.target

# SOLUTION (yes, we are cool)
print('Dataset size: \n{} posts in total'.format(len(x_train) + len(x_test)))
print('\t {} training posts'.format(len(x_train)))

for i in range(len(categories)):
  num = sum(y_train == i)
  print("\t\t {} {}".format(num, categories[i]))

print('\t {} testing posts'.format(len(x_test)))
for i in range(len(categories)):
  num = sum(y_test == i)
  print("\t\t {} {}".format(num, categories[i]))



# print('\n')
# print('EXAMPLE: \n')
# print(x_train[0])



Dataset size: 
3885 posts in total
	 2332 training posts
		 593 comp.windows.x
		 594 sci.med
		 599 soc.religion.christian
		 546 talk.politics.guns
	 1553 testing posts
		 395 comp.windows.x
		 396 sci.med
		 398 soc.religion.christian
		 364 talk.politics.guns


### 2.2 Cleaning of Data

In [5]:
# Remove lines starting with certain keywords 
def clean_post(post: str, remove_start: tuple):
    clean_lines = []
    for line in post.splitlines():
            if not line.startswith(remove_start):
                clean_lines.append(line)
    return '\n'.join(clean_lines)
    

# SOLUTION (yes, again, we are cool)
remove_start = (
  'From:',
  'Subject:',
  'Reply-To:',
  'In-Reply-To:',
  'Nntp-Posting-Host:',
  'Organization:',
  'X-Mailer:',
  'In article <',
  'Lines:',
  'NNTP-Posting-Host:',
  'Summary:',
  'Article-I.D.:'
)
x_train = [clean_post(p, remove_start) for p in x_train]
x_test = [clean_post(p, remove_start) for p in x_test]


## 3.TOKENISATION

In [ ]:
from transformers import DistilBertTokenizer

MAX_LEN = 512

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', padding=True, truncation=True)

# Let's check out how the tokenizer works
for n in range(3):
    # Tokenize forum post
    tokenizer_out = tokenizer(x_train[n])
    # Convert numerical tokens to alphabetical tokens
    encoded_tok = tokenizer.convert_ids_to_tokens(tokenizer_out.input_ids)
    # Decode tokens back to string
    decoded = tokenizer.decode(tokenizer_out.input_ids)
    print(tokenizer_out)
    print(encoded_tok, '\n')
    print(decoded, '\n')
    print('---------------- \n')



In [7]:
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 512

class PostsDataset(Dataset):
    def __init__(self, posts, labels, tokenizer, max_len):
        # Variables that are set when the class is instantiated
        self.posts = posts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.posts)
  
    def __getitem__(self, item):
        # Select the post and its category
        post = str(self.posts[item])
        label = self.labels[item]
        # Tokenize the post
        tokenizer_out = self.tokenizer(
            post,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
            )
        # Return a dictionary with the output of the tokenizer and the label
        return  {
            'input_ids': tokenizer_out['input_ids'].flatten(),
            'attention_mask': tokenizer_out['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Instantiate two PostsDatasets
train_dataset = PostsDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = PostsDataset(x_test, y_test, tokenizer, MAX_LEN)

## 4.MODEL

In [ ]:

from transformers import DistilBertModel

PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
distilbert = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
 
first_post = train_dataset[0]

hidden_state = distilbert(
    input_ids=first_post['input_ids'].unsqueeze(0),
    attention_mask=first_post['attention_mask'].unsqueeze(0)
    )

print(hidden_state[0].shape)
print(distilbert.config)



In [ ]:
from transformers import DistilBertPreTrainedModel, DistilBertConfig
import torch.nn.functional as F


PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'

class DistilBertForPostClassification(DistilBertPreTrainedModel):
    def __init__(self, config, num_labels, freeze_encoder=False):
        # Instantiate the parent class DistilBertPreTrainedModel
        super().__init__(config)
        # Instantiate num. of classes
        self.num_labels = num_labels
        # Instantiate and load a pretrained DistilBERT model as encoder
        self.encoder = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        # Freeze the encoder parameters if required (Q1)
        if freeze_encoder:
          for param in self.encoder.parameters():
              param.requires_grad = False
        
        # SOLUTION 3
        # An extra proposed layer
        self.ExtraLayer = torch.nn.Linear(
            in_features=config.dim, out_features=(config.dim-32), bias=True)

        # The classifier: a feed-forward layer attached to the encoder's head
        self.classifier = torch.nn.Linear(
            in_features=(config.dim-32), out_features=self.num_labels, bias=True)
        # Instantiate a dropout function for the classifier's input
        self.dropout = torch.nn.Dropout(p=0.1)

        


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        # Encode a batch of sequences with DistilBERT
        encoder_output = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        # Extract the hidden representations from the encoder output
        hidden_state = encoder_output[0]  # (bs, seq_len, dim)
        # Only select the encoding corresponding to the first token
        # of each sequence in the batch (Q2)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        # Apply dropout
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        
        # SOLUITION 3
        # Feed into our extra layer
        pooled_output = self.ExtraLayer(pooled_output)
        # Pass it through the ReLU function
        pooled_output = F.relu(pooled_output)

        # Feed into the classifier
        logits = self.classifier(pooled_output)  # (bs, dim)

        outputs = (logits,) + encoder_output[1:]
        
        if labels is not None: # (Q3)
          # Instantiate loss function
          # SOLUTION :
          loss_fct = torch.nn.CrossEntropyLoss()
          # Calculate loss
          # SOLUTION :
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
          # Aggregate outputs
          outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


# Instantiate model
model = DistilBertForPostClassification(
    config=distilbert.config, num_labels=len(categories), freeze_encoder = True
    )

model_unfreezed = DistilBertForPostClassification(
    config=distilbert.config, num_labels=len(categories), freeze_encoder = False)

# Print info about model's parameters
total_params = sum(p.numel() for p in model.parameters())
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])
print('Model total params: ', total_params)
print('Model trainable params: ', trainable_params)
print('\n', model)

## 5.TRAINING

In [ ]:

from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',          
    logging_dir='./logs',
    logging_first_step=True,
    logging_steps=50,
    num_train_epochs=10,              
    per_device_train_batch_size=8,  
    learning_rate=5e-5,
    weight_decay=0.01        
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

trainer_unfreezed = Trainer(
    model=model_unfreezed,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

# Freeze version
# train_results = trainer.train()
# test_results = trainer.predict(test_dataset=test_dataset)

# Unfreeze version
train_results = trainer_unfreezed.train()
test_results = trainer_unfreezed.predict(test_dataset=test_dataset)


In [11]:
print('Predictions: \n', test_results.predictions)
print('\nAccuracy: ', test_results.metrics['test_accuracy'])
print('Precision: ', test_results.metrics['test_precision'])
print('Recall: ', test_results.metrics['test_recall'])
print('F1: ', test_results.metrics['test_f1'])
print(categories)

MODEL_PATH = './my_model'
trainer.save_model(MODEL_PATH)

Saving model checkpoint to ./my_model
Configuration saved in ./my_model/config.json


Predictions: 
 [[ 8.151242  -2.8278337 -3.309761  -2.6220899]
 [ 8.174492  -2.9332068 -3.2482893 -2.5959346]
 [ 8.181815  -2.9110806 -3.2325187 -2.6634274]
 ...
 [-2.9325624 -3.0623033 -2.724237   7.6076756]
 [-3.140176  -3.2823186 -2.590203   7.8240466]
 [-3.1848192 -3.0800579  8.339697  -2.9060147]]

Accuracy:  0.9632968448164843
Precision:  [0.96992481 0.98123324 0.93160377 0.97478992]
Recall:  [0.97974684 0.92424242 0.99246231 0.95604396]
F1:  [0.97481108 0.95188557 0.96107056 0.96532594]
['comp.windows.x', 'sci.med', 'soc.religion.christian', 'talk.politics.guns']


Model weights saved in ./my_model/pytorch_model.bin


## 6.PREDICTIONS

In [ ]:
device = "cpu"

model = DistilBertForPostClassification.from_pretrained(
    './my_model', config=distilbert.config, num_labels=len(categories)).to(device)
for sentence in ['Lung cancer is a deadly disease.', 'God is love', 'How can you install Microsoft Office extensions?', 'Gun killings increase every year.']:
  encoding = tokenizer.encode_plus(sentence)
  encoding['input_ids'] = torch.tensor([encoding.input_ids]).to(device)
  encoding['attention_mask'] = torch.tensor(encoding.attention_mask).to(device)
  out = model(**encoding)
  categories_probability = torch.nn.functional.softmax(out[0], dim=1).flatten()
  print(sentence)
  print('\tProbabilities assigned by the model : ')
  for n,c in enumerate(categories):
    print('\t\t{} : {}'.format(c, categories_probability[n]))
  print('\n\t--> Prediction :', categories[categories_probability.argmax()])
  print('------------------------------------------------\n')
  

## 7.IMPROVE THE MODEL

In [ ]:

# SOLUTION 1 (trivial): increase training epochs
# SOLUTION 2: finetune encoder parameters too

# model_unfreezed = DistilBertForPostClassification(config, freeze_decoder = False)
# trainer_unfreezed = Trainer(
#     model=model_unfreezed,                         
#     args=training_args,                  
#     train_dataset=train_dataset,         
#     compute_metrics=compute_metrics
# )
# trainer_unfreezed.train()
# trainer_unfreezed.predict(test_dataset=test_dataset)

# # SOLUTION 3: let's see what students can do !


# REPORT

## SOLUTION 0

With the default given parameters:

```py
num_train_epochs=4,
per_device_train_batch_size=8,
learning_rate=5e-5,
weight_decay=0.01
```


* Accuracy = 0.9001931745009659
* Precision = [0.89277389 0.88235294 0.9 0.93030303]
* Recall = [0.96962025 0.83333333 0.94974874 0.84340659]
* F-score = [0.92961165 0.85714286 0.92420538 0.88472622]

## SOLUTION 1

We increase the number of training epochs to 10

```py
num_train_epochs=10,
per_device_train_batch_size=8,
learning_rate=5e-5,
weight_decay=0.01
```

* Accuracy:  0.9336767546683837
* Precision:  [0.93658537 0.96111111 0.92909535 0.90909091]
* Recall:  [0.9721519  0.87373737 0.95477387 0.93406593]
* F1:  [0.95403727 0.91534392 0.9417596  0.92140921]

We can see that there has been a slight improvement in terms of accuracy as well as F-score when increasing the epochs to 10. Increasing beyond 10 *might* increase the scores little bit, but instead of doing that, let's take a look at some other solutions also.

## SOLUTION 2

This time, we keep the hyperparameters (specifically the no. of epochs) the same, but we unfreeze the encoder parameters.

So in order to do so, we insert the folliwng snippet near the end of section "4. MODEL":

```py
model_unfreezed = DistilBertForPostClassification(
    config=distilbert.config, num_labels=len(categories), freeze_encoder = False)
```


And this in section "5. TRAINING"

```py
trainer_unfreezed = Trainer(
    model=model_unfreezed,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

# Freeze version
# train_results = trainer.train()
# test_results = trainer.predict(test_dataset=test_dataset)

# Unfreeze version
trainer_unfreezed.train()
test_results = trainer_unfreezed.predict(test_dataset=test_dataset)
```



As before, we kep our hyperparameters the same as previous solution
```py
num_train_epochs=10,
per_device_train_batch_size=8,
learning_rate=5e-5,
weight_decay=0.01
```
And so, we get the following results: 

* Accuracy:  0.9639407598197038
* Precision:  [0.96766169 0.98395722 0.92957746 0.98005698]
* Recall:  [0.98481013 0.92929293 0.99497487 0.94505495]
* F1:  [0.9761606  0.95584416 0.96116505 0.96223776]

We see that unfreezing the parameters for encoder does improve the accuracy as well as F1 score.


## SOLUTION 3

As a first proposal, we can try to increase the number of layers in the model. We' re gonna try this with both freeze and unfreeze method.

In the "4. MODEL" section, The following snippets were added:

```py
# SOLUTION 3
# An extra proposed layer
self.ExtraLayer = torch.nn.Linear(
    in_features=config.dim, out_features=(config.dim-32), bias=True)
.
.
.
# SOLUTION 3
# Feed into our extra layer
pooled_output = self.ExtraLayer(pooled_output)
# Pass it through the ReLU function
pooled_output = F.relu(pooled_output)
```

| Epochs | Freeze | Linear Layers |          P          |          R          |          F          |  Acc |
|:------:|:------:|:-------------:|:-------------------:|:-------------------:|:-------------------:|:----:|
|   10   |   Yes  |    1 Extra    | 0.94 0.96 0.93 0.93 | 0.96 0.90 0.97 0.93 | 0.95 0.93 0.95 0.93 | 0.94 |
|   10   |   No   |    1 Extra    | 0.96 0.98 0.93 0.97 | 0.97 0.92 0.99 0.95 | 0.97 0.95 0.96 0.96 | 0.96 |
|        |        |               |                     |                     |                     |      |

We observe that simply adding 1 linear layer improves the model a bit if we compare the 1st row with the SOLUTION 1 proposed earlier.
But upon unfreezing the parameters, having this extra linear layer gives no additional benefit, it we comapre this 2nd row in the table with our previous SOLUTION 2.


## Conclusion

This BERT practical assignment gave us a hands-on experience of using the pre-trained transformer models provided by Hugging Face. Training the model with the best hyperparameters is tricky process but not an impossible one. Some bruth force techniques like Grid Search can be utilized in order to find the best hyperparameters.

Possible extension of this assignment:

* Use of a different model (like RoBERTa or GPT-2) and compare the results

---